# Permutations of a Simple Circuit

This notebook walks through how to utilize the core semantics of SysML v2 to generate alternative circuits as inputs to an OpenMDAO solution of these circuits. 

## Background

The M1 user model in SysML v2 is meant to be a set of constraints and rules under which legal instances can be created. Those instances should be taken as alternative produced systems and they can be analyzed in that way.

## Libraries Load-Up

Load up PyMBE and its various libraries.

In [ ]:
NUM_INTERPRETATIONS = 50
NUM_RESISTORS = (2, 12)
NUM_DIODES = (1, 2)
NUM_CONNECTORS = (15, 30)

In [ ]:
from pathlib import Path
import networkx as nx
import matplotlib as plt

import pymbe.api as pm

from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
    random_generator_phase_1_multiplicities,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Load Up Model

Read the model from the local JSON file.

In [ ]:
circuit_file = Path(pm.__file__).parent / "../../tests/fixtures/Circuit Builder.json"

circuit_model = pm.Model.load_from_file(circuit_file)
circuit_lpg = SysML2LabeledPropertyGraph(model=circuit_model)
[id_to_circuit_name_lookup, circuit_name_to_id_lookup] = build_stable_id_lookups(circuit_lpg)

circuit_lpg.model.MAX_MULTIPLICITY = 100

## Explore Contents of Model with M1 in Memory

Use the M1 memory objects to see what is in the current model, starting with the main packages.

In [ ]:
circuit_model.packages

In [ ]:
circuit_model.ownedElement["Circuit Builder"].ownedElement

In [ ]:
circuit_def = circuit_model.ownedElement["Circuit Builder"].ownedElement["Circuit"]

### Circuit and its Features

Here is the circuit and its features, both parts and used connections.

In [ ]:
circuit_def.relationships

In [ ]:
circuit_def.ownedMember

## Update multiplicities
for `Resistors`, `Diodes`, and `Connections`

In [ ]:
multiplicity = circuit_def.ownedMember["Circuit Resistor"].multiplicity
multiplicity.lowerBound._data["value"], multiplicity.upperBound._data["value"] = NUM_RESISTORS

multiplicity = circuit_def.ownedMember["Circuit Diode"].multiplicity
multiplicity.lowerBound._data["value"], multiplicity.upperBound._data["value"] = NUM_DIODES

# Get the ConnectionUsage
for member in circuit_def.ownedMember:
    if member._metatype != "ConnectionUsage":
        continue
    member.multiplicity.lowerBound._data["value"], member.multiplicity.upperBound._data["value"] = NUM_CONNECTORS
    connection = member

## Generate M0 instances from the M1 model

Use the M1 model to start creating a series of instances to represent the circuits that should be analyzed.

In [ ]:
m0_interpretations = [
    random_generator_playbook(
        lpg=circuit_lpg,
        name_hints={},
        filtered_feat_packages=[circuit_lpg.model.ownedElement["Circuit Builder"]],
        phase_limit=10,
    ) for _ in range(NUM_INTERPRETATIONS)
]

### Sort the interpretations by number of connections
Sorted from `most` to `least`, and pick the first one.

In [ ]:
# sort the interpretations from most connections to fewer connections
m0_interpretations = [*sorted(m0_interpretations, key=lambda x: len(x[connection._id]), reverse=True)]
m0_interpretation = m0_interpretations[0]

## Filter M0 Instances for Reasonable Circuits

Until we get more sophisticated and can interpret constraints, the initial approach is to filter out solutions with unanalyzable layouts or trim the layouts to something more tractable.

### Connector End Checks

Look at the ends of the three main kinds of connectors.

In [ ]:
p2p = circuit_def.ownedMember["Part to Part"]
p2p.endFeature[0]._id

In [ ]:
source_feat, target_feat = p2p.endFeature
for source, target in zip(m0_interpretation[source_feat._id], m0_interpretation[target_feat._id]):
    print(source, "-->", target)

> Janky A.F. connector filter

The cell below filters out:
* self-connections
* duplicate connectors between the same m0 pins

In [ ]:
import ipywidgets as ipyw
import matplotlib.pyplot as plt
from IPython.display import display


def get_unique_connections(instances, feature):
    source_feat, target_feat = feature
    m0_connector_ends = [
        (tuple(source), tuple(target))
        for source, target in zip(instances[source_feat._id], instances[target_feat._id])
    ]

    m0_connector_ends = tuple({
        (source, target)
        for source, target in m0_connector_ends
        if source[:-1] != target[:-1]
    })

    unique_connections = {
        (source[-2:], target[-2:]): (source, target)
        for source, target in m0_connector_ends
    }
    return tuple((source, target) for source, target in unique_connections.values())
    return tuple((source, target) for source, target in m0_connector_ends)


def draw_circuit(circuit_graph: nx.DiGraph, figsize=None, rad=0.1, arrowsize=40, linewidth=2, layout="kamada_kawai"):
    figsize = figsize or (20, 20)

    color_dict = {
        "blue": "R",
        "#A020F0": "D",
        "red": "EMF",
    }
    layout_algorithm = getattr(nx.layout, f"{layout}_layout")
    node_pos = layout_algorithm(circuit_graph)

    internal_edge_list = [
        (n1, n2)
        for n1, n2 in circuit_graph.edges
        if n1[0] == n2[0]
    ]
    connect_edge_list = [
        edge
        for edge in circuit_graph.edges
        if edge not in internal_edge_list
    ]

    plt.figure(figsize=figsize)
    for color, colored_nodes in color_dict.items():
        poses = {
            node: loc
            for node, loc in node_pos.items() if node[0].startswith(colored_nodes)
        }
        nx.draw_networkx_nodes(circuit_graph, poses, nodelist=list(poses.keys()), node_size=1200, node_color=color)
        
    nx.draw_networkx_edges(circuit_graph, node_pos, edgelist=connect_edge_list, edge_color="black", width=linewidth, arrowsize=arrowsize, connectionstyle=f"arc3,rad={rad}")
    nx.draw_networkx_edges(circuit_graph, node_pos, edgelist=internal_edge_list, edge_color="black", width=linewidth, style="-.", arrowsize=arrowsize, connectionstyle=f"arc3,rad={rad}")

    labels = {(node, polarity): f"{node}{'-' if polarity=='Neg' else '+'}" for node, polarity in node_pos}
    label_options = {"boxstyle": "circle", "ec": "white", "fc": "white", "alpha": 0.0}

    nx.draw_networkx_labels(
        circuit_graph,
        node_pos,
        labels,
        font_size=8,
        font_color="white",
        font_weight="bold",
        bbox=label_options,
    )
    plt.show()

nx_layouts = sorted([
    layout.replace("_layout", "")
    for layout in dir(nx.layout)
    if layout.endswith("_layout")
    and not any(bad_stuff in layout for bad_stuff in ("partite", "rescale", "planar"))
])
    
@ipyw.interact()
def draw_graph(interpretation=(0, len(m0_interpretations)-1), edge_curvature=(0, 0.5, 0.05), linewidth=(1, 3, 0.2), layout=nx_layouts):
    unique_connections = get_unique_connections(m0_interpretations[interpretation], p2p.endFeature)

    def get_el_name(element):
        name = element.name
        name, num = name.split("#")
        name = name if name == "EMF" else name[0]
        return name + num

    graph = nx.DiGraph()
    edges = [
        ((get_el_name(source[-2]), "Pos"), (get_el_name(target[-2]), "Neg"))
        for source, target in unique_connections
    ]
    nodes = {
        n[0]
        for n, _ in edges
        for _, n in edges
    }
    edges += [
        ((node, "Neg"), (node, "Pos"))
        for node in nodes
        if not node.startswith("EMF")
    ]
    graph.add_edges_from(edges)
    draw_circuit(graph, figsize=(20, 10), rad=edge_curvature, linewidth=linewidth, layout=layout)

# OpenMDAO
> Based on OpenMDAO's [nonlinear circuit analysis example](https://openmdao.org/newdocs/versions/latest/examples/circuit_analysis_examples.html).

In [ ]:
from importlib import import_module

import networkx as nx
import openmdao.api as om

## Load OpenMDAO component classes

In [ ]:
def load_class(class_path: str) -> type:
    *module_path, class_name = class_path.split(".")
    module = import_module(".".join(module_path))
    return getattr(module, class_name)

In [ ]:
# TODO: this should be retrieved from the SysML model
components_to_om = {
    "Diode": "openmdao.test_suite.test_examples.test_circuit_analysis.Diode",
    "Resistor": "openmdao.test_suite.test_examples.test_circuit_analysis.Resistor",
    "Node": "openmdao.test_suite.test_examples.test_circuit_analysis.Node",
}

components_to_om = {
    name: load_class(class_path)
    for name, class_path in components_to_om.items()
}

# FIXME: figure out a way to generalize this patterns
circuit_components = {
    element.name: dict(id=element._id, om=components_to_om[element.name])
    for element in circuit_model.ownedElement["Circuit Builder"].ownedElement
    if element.name in components_to_om
}

## Declare an OpenMDAO Group based on the SysML Model

In [ ]:
class Circuit(om.Group):
    
    def initialize(self):
        self.options.declare("interpretation", types=dict)

    def setup(self):
        interpretation = self.options["interpretation"]
        self.m0_to_om_components = component_map = {}

        unique_connections = get_unique_connections(interpretation, p2p.endFeature)
        digraph = nx.DiGraph()
        digraph.add_edges_from((source[-2:], target[-2:]) for source, target in unique_connections)

        internal_edges = {}
        for src, tgt in digraph.edges:
            src_el, src_pin = src
            tgt_el, tgt_pin = tgt
            for el in (src_el, tgt_el):
                if el not in internal_edges:
                    internal_edges[el] = {}
            internal_edges[src_el]["out"] = src_pin
            internal_edges[tgt_el]["in"] = tgt_pin

        full_digraph = digraph.copy()
        full_digraph.add_edges_from(
            ((el, pins["in"]), (el, pins["out"]))
            for el, pins in internal_edges.items()
            if not el.name.lower().startswith("emf")
            and "in" in pins and "out" in pins
        )

        emf = [el for el in internal_edges if el.name.lower().startswith("emf")][0]
        emf_out = (emf, internal_edges[emf]["out"])
        emf_in = (emf, internal_edges[emf]["in"])

        valid_items = {
            node[0]
            for path in nx.all_simple_paths(full_digraph, emf_out, emf_in)
            for node in path
        }
        V = next(digraph.successors(emf_out))
        Vg = next(digraph.predecessors(emf_in))

        for item_type in ("Resistor", "Diode"):
            for idx, item_sequence in enumerate(interpretation[circuit_components[item_type]["id"]]):
                item = item_sequence[-1]
                if item not in valid_items:
                    print(f"Skipping {item} ({idx})")
                    continue
                kwargs = {}
                if item in Vg:
                    print(">>> Promoting Vg based on:", item)
                    kwargs["promotes_inputs"] = [('V_out', 'Vg')]
                component_map[item] = self.add_subsystem(
                    str(item).replace("#", "_").lower(),
                    components_to_om[item_type](
                        # TODO: parse the resistor/diode parameters and pass them as keyword arguments them here
                    ),
                    **kwargs,
                )

        for node_id, comp in enumerate(nx.connected_components(digraph.to_undirected())):
            comp_edges = [*nx.subgraph(digraph, list(comp)).edges]
            sources = [edge[0] for edge in comp_edges if edge[0][0] in valid_items]
            targets = [edge[1] for edge in comp_edges if edge[1][0] in valid_items]

            num_sources = sum(1 for node in comp if node in sources)
            num_targets = sum(1 for node in comp if node in targets)

            node_name = f"node_{node_id}"
            kwargs = {}
            if V in sources + targets:
                print(">>> Promoting I_in based on:", node_name)
                kwargs["promotes_inputs"] = [("I_in:0", "I_in")]
                kwargs["promotes_outputs"] = [("V", "V")]
            node = self.add_subsystem(
                node_name,
                components_to_om["Node"](n_in=num_sources, n_out=num_targets),
                **kwargs,
            )
            indeces = {"in": 0, "out": 0}
            elec_volt_pins = []
            for elec_pin in comp:
                is_source = elec_pin in sources
                elec_el, pin = elec_pin[-2:]
                if elec_el not in valid_items:
                    continue
                elec_el_name = str(elec_el).replace("#", "_").lower()
                if elec_el_name.startswith("emf"):
                    continue
                elec_dir = "out" if is_source else "in"
                node_dir = "in" if is_source else "out"
                elec_volt_pins += [f"{elec_el_name}.V_{elec_dir}"]
                self.connect(f"{elec_el_name}.I", f"{node_name}.I_{node_dir}:{indeces[node_dir]}")
                indeces[node_dir] += 1
            if elec_volt_pins:
                try:
                    self.connect(f"{node_name}.V", elec_volt_pins)
                except:
                    print(">>>", f"{node_name}.V", elec_volt_pins)

        self.nonlinear_solver = om.NewtonSolver()
        self.linear_solver = om.DirectSolver()

        self.nonlinear_solver.options['iprint'] = 2
        self.nonlinear_solver.options['maxiter'] = 10
        self.nonlinear_solver.options['solve_subsystems'] = True
        self.nonlinear_solver.linesearch = om.ArmijoGoldsteinLS()
        self.nonlinear_solver.linesearch.options['maxiter'] = 10
        self.nonlinear_solver.linesearch.options['iprint'] = 2

In [ ]:
for idx, interpretation in enumerate(m0_interpretations):
    try:
        p = om.Problem()
        model = p.model
        circuit_name = f'circuit_{idx}'
        model.add_subsystem(circuit_name, Circuit(interpretation=interpretation))
        p.setup()
        print(f"[{idx}] successfully ran setup")
        try:
            p[f'{circuit_name}.V'] = 10.
            p[f'{circuit_name}.Vg'] = 0.
            p[f'{circuit_name}.I_in'] = 0.1
            print(f"[{idx}] attempting run")
            p.run_model()
            print(f"[{idx}] kinda ran\n\n\n")
            break
        except KeyError as exc:
            print(exc)
        except Exception as exc:
            print(exc)
    except Exception as exc:
        print(exc)
        # raise
        #print(f"  > Failed to instantiate {idx}")
    break

In [ ]:
p.model.circuit_0.node_0._var_rel_names

In [ ]:
%debug

In [ ]:
print(idx)
om.view_connections(p)

In [ ]:
om.n2(p)

### Appendix

In [ ]:
unique_connections = get_unique_connections(m0_interpretation, p2p.endFeature)
digraph = nx.DiGraph()
digraph.add_edges_from((source[-2:], target[-2:]) for source, target in unique_connections)
digraph.edges

internal_edges = {}
for src, tgt in digraph.edges:
    src_el, src_pin = src
    tgt_el, tgt_pin = tgt
    for el in (src_el, tgt_el):
        if el not in internal_edges:
            internal_edges[el] = {}
    internal_edges[src_el]["out"] = src_pin
    internal_edges[tgt_el]["in"] = tgt_pin

digraph.add_edges_from(
    ((el, pins["in"]), (el, pins["out"]))
    for el, pins in internal_edges.items()
    if not el.name.lower().startswith("emf")
)

emf = [el for el in internal_edges if el.name.lower().startswith("emf")][0]
internal_edges[emf]
digraph.neighbors((emf, internal_edges[emf]["out"]))

limits = plt.axis("off")
plt.Figure(figsize=(20,20))
nx.draw_circular(digraph, with_labels=False)
plt.show()
list(digraph.successors((emf, internal_edges[emf]["out"]))), list(digraph.predecessors((emf, internal_edges[emf]["in"])))